In [1]:
import pandas as pd
import os
import numpy as np
folder_path = "Data"  # Make sure the folder path is correct

# Read each CSV file into a DataFrame
df_price = pd.read_csv(os.path.join(folder_path, "Prices.csv"))
df_chargesvar = pd.read_csv(os.path.join(folder_path, "Charges_var.csv"))
df_prod = pd.read_csv(os.path.join(folder_path, "Production.csv"))
df_plantation = pd.read_csv(os.path.join(folder_path, "plantation.csv"))
df_month_index = pd.read_csv(os.path.join(folder_path, "month_index.csv"))
df_sim=pd.read_csv(os.path.join(folder_path, "Simulation.csv"))
# Now you can work with these DataFrames as needed
df_price = df_price.fillna(0)
df_prod=df_prod.fillna(0)
serre_sau_dict = df_sim.set_index('serre')['SAU(ha)'].to_dict()


In [2]:
sum_last_37 = df_prod.iloc[:, -37:].sum(axis=1)

# Create a dictionary to associate scenario with the sum of the last 37 columns
scenario_prod = dict(zip(df_prod['Scénario'], sum_last_37))


In [3]:
scenario_variety_mapping = df_prod[['Scénario', 'variété 23-24']]

# Initialize an empty dictionary to store the mappings
variety_scenario_dict = {}

# Iterate through each row of the DataFrame
for index, row in scenario_variety_mapping.iterrows():
    scenario = row['Scénario']
    variety = row['variété 23-24']
    
    # If the variety is not already in the dictionary, add it with an empty list
    if variety not in variety_scenario_dict:
        variety_scenario_dict[variety] = []
    
    # Append the scenario to the list of scenarios for this variety
    variety_scenario_dict[variety].append(scenario)

# Print the dictionary
print(variety_scenario_dict)

{'YAZMIN': [1, 2, 3, 4, 5, 15], 'Adelita': [8, 9, 10], 'VICTORIA': [11], 'LAURITA': [12, 13, 14], 'Clara': [18], 'Henrienta': [19]}


In [4]:
import pandas as pd
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')

# Define the start date
start_date = datetime(2024, 1, 1)

# Function to get the month index
def get_month_index(month):
    month_dict = {
        'Janvier': 1,
        'Février': 2,
        'Mars': 3,
        'Avril': 4,
        'Mai': 5,
        'Juin': 6,
        'Juillet': 7,
        'Aout': 8,
        'Septembre': 9,
        'Octobre': 10,
        'Novembre': 11,
        'Décembre': 12
    }
    return month_dict[month]

# Generate the data
data = []
months = [
    'Janvier', 'Février', 'Mars', 'Avril', 'Mai', 'Juin', 'Juillet', 'Aout', 'Septembre', 'Octobre', 'Novembre', 'Décembre'
]

num_weeks=84

for index in range(1,num_weeks+1):
    month_index = start_date.month
    month_name = months[month_index - 1]  # Adjust for zero-based indexing

# Capitalize the first letter of the month name
    month = month_name.capitalize()

    # month = start_date.strftime('%B').capitalize()
    month_index = get_month_index(month)
    data.append([month, start_date.strftime('%d/%m/%Y'), month_index, index])
    start_date += timedelta(weeks=1)

# Create DataFrame
df_week_index = pd.DataFrame(data, columns=['Month', 'Date', 'Month Index', 'Week Index'])

# Save to CSV
month_to_week_indices = {}

# Iterate through the DataFrame to populate the dictionary
for index, row in df_week_index.iterrows():
    month = row['Month']
    week_index = row['Week Index']
    
    if month not in month_to_week_indices:
        month_to_week_indices[month] = []
    
    month_to_week_indices[month].append(week_index)

# Print the dictionary


In [5]:
scenario_delai_dict = {}
scenario_duree_dict = {}
scenario_mois_dict = {}
scenario_culture={}
scenario_cmo={}
scenario_vitesse={}
scenario_cv={}
for index,row in df_chargesvar.iterrows():
    scenario = row['Scénario']
    cmo=row["Coûts de Main d'œuvre par kg de production"]
    vitesse=row["Vitesse de main d'œuvre kg/personne/jour"]
    cv=row["Coûts variable par hectar"]
    scenario_cmo[scenario]=cmo
    scenario_vitesse[scenario]=vitesse
    scenario_cv[scenario]=cv
for index, row in df_prod.iterrows():
    scenario = row['Scénario']
    delai = row['Délai pour début de production']
    duree = row['Durée de production en semaine']
    mois = row['Mois']
    culture=row['Culture']
    
    scenario_delai_dict[scenario] = delai
    scenario_duree_dict[scenario] = duree
    scenario_mois_dict[scenario] = mois
    scenario_culture[scenario] =culture
    
scenarios=list(scenario_culture.keys())


In [6]:
price={}
price["Framboise"]=np.array(df_price.iloc[0,1:])
price["Mure"]=np.array(df_price.iloc[1,1:])


In [7]:
secteur_serre_dict = {}

# Iterate over the rows of the DataFrame
for index, row in df_sim.iterrows():
    secteur = row['secteur']
    serre = row['serre']
    
    # If the secteur is not in the dictionary yet, create a new list
    # Otherwise, append the serre to the existing list
    if secteur not in secteur_serre_dict:
        secteur_serre_dict[int(secteur)] = [int(serre)]
    else:
        secteur_serre_dict[int(secteur)].append(int(serre))
serre_secteur_dict = {serre: secteur for secteur, serres in secteur_serre_dict.items() for serre in serres}
num_serre=df_sim.shape[0]
num_sect=len(secteur_serre_dict)


In [8]:
import numpy as np
prod_mat=np.matrix(df_prod.iloc[:,8:])

# Assuming num_serre, num_scenario, and prod_mat are defined somewhere
# Assuming price, scenario_culture, scenario_delai_dict, and scenario_duree_dict are also defined

# Initialize prod tensor
prod = np.zeros((num_serre, len(scenarios), num_weeks))

def padded_dot(a, b):
    # Get the lengths of the arrays
    len_a = a.shape[1]
    len_b = b.shape[1]
    
    # Determine the maximum length
    max_len = max(len_a, len_b)
    
    # Pad arrays with zeros
    a_padded = np.pad(a, ((0, 0), (0, max_len - len_a)), mode='constant')
    b_padded = np.pad(b, ((0, 0), (0, max_len - len_b)), mode='constant')
    
    # Perform dot product
    result = np.dot(a_padded, b_padded.T)
    
    return result

   #Attention ici et en dessous
for i in range(num_serre):
    for j in scenarios:
        for t in range(num_weeks):
            if t in month_to_week_indices[scenario_mois_dict[j]]:
                if t + scenario_delai_dict[j] + scenario_duree_dict[j]<=89:
                   
                   price_array = np.array(price[scenario_culture[j]][t + scenario_delai_dict[j]:
                                                               t + scenario_delai_dict[j] 
                                                               + scenario_duree_dict[j]])
                   prod_mat_array = np.array(prod_mat[scenarios.index(j),  :scenario_duree_dict[j]])
                   prod[(i, scenarios.index(j), t)] = np.dot(price_array.reshape(1, -1), prod_mat_array.reshape(-1, 1))[0, 0]
                else:
                    prod[(i, scenarios.index(j), t)] = padded_dot(
    np.array(price[scenario_culture[j]][t + scenario_delai_dict[j]:]).reshape(1, -1),
    np.array(prod_mat[scenarios.index(j), :]).reshape(1, -1)
)
  


            # Calculate dot product using numpy's dot function 

In [35]:
import gurobipy as gp
from gurobipy import GRB

try:
    # Create a new model
    m = gp.Model("portfolio")

    
    choices={}
    for i in range(num_serre):
        for j in scenarios:
            for t in month_to_week_indices[scenario_mois_dict[j]]:
                choices[(i,j,t)]= m.addVar(vtype=GRB.BINARY, name=f'choice_{i}_{j}_{t}')
    
   
    for i in range(num_serre):
        m.addConstr(gp.quicksum(choices[(i, j, t)] for j in scenarios for t in month_to_week_indices[scenario_mois_dict[j]]) == 1, f"constraint_{i}")
   
    # Meme scenario par secteur
    for i in range(num_sect):
        ref = secteur_serre_dict[i+1][0]
        # m.addConstr(gp.quicksum(choix_semaines[(i, t)] for t in range(90)))
        for j in secteur_serre_dict[i+1]:
            for k in scenarios:
                for t in month_to_week_indices[scenario_mois_dict[k]]:
                    m.addConstr(choices[(j-1,k,t)] 
                                == choices[(ref-1,k,t)], f'c_0_{j}_{k}_{t}')
                

    # Cette contrainte est obtenue par la contraposé de la cinquième contrainte
    for i in range(num_serre):
        for j in scenarios:
            if j==5 and i!=5:
                for t in month_to_week_indices[scenario_mois_dict[j]]:
                    m.addConstr(choices[(i,j,t)]==0)

    #Contrainte limite de main d'oeuvre
    
    #contrainte choix
    for i in range(num_serre):
        if i!=14 or i!=17:
            for j in variety_scenario_dict["Clara"]:
                for t in month_to_week_indices[scenario_mois_dict[j]]:
                    m.addConstr(choices[(i,j,t)]==0)
            for j in variety_scenario_dict["LAURITA"]:
                for t in month_to_week_indices[scenario_mois_dict[j]]:
                    m.addConstr(choices[(i,j,t)]==0)
        else:
            if serre_sau_dict[i+1]>2.87:
                for j in variety_scenario_dict["Clara"]:
                    for t in month_to_week_indices[scenario_mois_dict[j]]:
                        m.addConstr(choices[(i,j,t)]==0)
                for j in variety_scenario_dict["LAURITA"]:
                    for t in month_to_week_indices[scenario_mois_dict[j]]:
                        m.addConstr(choices[(i,j,t)]==0)
    #for s in range(20,89):
                        
    # for t such that:
        #conditions:
    #t in month_to_week_indices[scenario_mois_dict[j]]
    #month_to_week_indices[scenario_mois_dict[j]])+scenario_delai_dict[j]) less than k
    #month_to_week_indices[scenario_mois_dict[j]])+scenario_delai_dict[j])+scenario_durre_dict[j] greater than k
            
    #sum(choices(i,j,t)*prod(t+delai+k)*sau/vitesse for i in serre for j in scenarios if t and j satisfies #for k in range(durre prod))
    #conditions:
    #t in month_to_week_indices[scenario_mois_dict[j]]
    #month_to_week_indices[scenario_mois_dict[j]])+scenario_delai_dict[j]) less than k
    #month_to_week_indices[scenario_mois_dict[j]])+scenario_delai_dict[j])+scenario_durre_dict[j] greater than k
                        

    for s in range(20,90):
        scenarios_time_dict={}
        for j in scenarios:
            scenarios_time_dict[j]=[]
            for t in month_to_week_indices[scenario_mois_dict[j]]:
                if t+scenario_delai_dict[j]<s and s<t+scenario_delai_dict[j]+scenario_duree_dict[j]:
                    scenarios_time_dict[j].append(t)
                else: 
                    pass
        m.addConstr(gp.quicksum(choices[(i,k,t)]*(serre_sau_dict[i+1]/scenario_vitesse[k])
                                *prod_mat[scenarios.index(k),s-t-scenario_delai_dict[k]-1]
                                  for i in range(num_serre) for k in scenarios for t in scenarios_time_dict[k])<=600), f'mo_{s}'




    CA_expr = gp.quicksum( choices[(i,j,t)]* prod[(i, scenarios.index(j), t-1)] * serre_sau_dict[i+1]
                     for j in scenarios for t in month_to_week_indices[scenario_mois_dict[j]]
                        for i in range(num_serre))
    CMO_expr = gp.quicksum(choices[(i,j,t)] * scenario_prod[j]
                        #    can be changed 
                           * serre_sau_dict[i+1] * scenario_cmo[j] 
                       for j in scenarios for t in month_to_week_indices[scenario_mois_dict[j]]
                        for i in range(num_serre))
    CV_expr = gp.quicksum(choices[(i,j,t)]* 
                          serre_sau_dict[i+1] * 
                          scenario_cv[j] for j in scenarios for t in month_to_week_indices[scenario_mois_dict[j]]
                        for i in range(num_serre))

    # Set objective
    m.update()
    m.setObjective(CA_expr - CV_expr- CMO_expr, GRB.MAXIMIZE)

    # Optimize model
    m.optimize()

    # for v in m.getVars():
    #     print(f"{v.VarName} {v.X:g}")
    # After the optimization is done, you can print the expressions
    print("Value of CA_expr:", CA_expr.getValue())
    print("Value of CMO_expr:", CMO_expr.getValue())
    print("Value of CV_expr:", CV_expr.getValue())


    print(f"Obj: {m.ObjVal:g}")

except gp.GurobiError as e:
    print(f"Error code {e.errno}: {e}")

except AttributeError:
    print("Encountered an attribute error")


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-7300U CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 2965 rows, 2280 columns and 23715 nonzeros
Model fingerprint: 0x37444cdf
Variable types: 0 continuous, 2280 integer (2280 binary)
Coefficient statistics:
  Matrix range     [5e-01, 5e+02]
  Objective range  [1e+03, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+02]
Found heuristic solution: objective -2.42498e+07
Presolve removed 2941 rows and 2233 columns
Presolve time: 0.06s
Presolved: 24 rows, 47 columns, 418 nonzeros
Found heuristic solution: objective -1.62137e+07
Variable types: 0 continuous, 47 integer (47 binary)

Root relaxation: objective -1.616180e+07, 26 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj

In [26]:
print("s:", s)
print("t:", t)
print("scenario_delai_dict[k]:", scenario_delai_dict[k])
print("prod_mat length:", len(prod_mat))


s: 20
t: 44
scenario_delai_dict[k]: 12
prod_mat length: 15
